## 简介
介绍三种自动测量MT质量的方法。其中一种是通过计算评估分数，而另外两种是基于依赖结构。<br>
- AutoDA: 一个基于语义属性的线性拟合模型，使用WMT Assessment scores或HUMEseg scores进行训练
- Tree Aggreg: 基于N-gram的通过对齐的句法结构进行测量的一种评分标准N-gram based metric computed over aligned syntactic structures instead of the linear representation of the translated sentences，这个没看懂？
- NMTScorer: 是一个序列分类器，通过返回0和1之间的一个数，来表示MT的好坏程度。（把MT的译文和对应Reference分别放入LSTM，比对结果输出的相识性）

下面大部分是原文、、、、、、
## AutoDA: Automatic Direct Assessment
AutoDA是一个简单的线性回归模型。他使用的属性从MT译文和reference对中提取出来。这其中可能包含了一些其他metrics中使用的属性。The metric is based on a simple linear regression combining several features extracted fromthe automatically aligned(什么叫自动对齐了的？？？) trainslation-reference pair. There may be also other established metrics within the features.<br>
这里会介绍两种autoDA
### AutoDA Using Czech Tectogrammatics
这种方法会自动分析Czech的翻译和对应的reference翻译，并从中提取属性，计算并给出分数。
#### Word Alignment
要使用AutoDA，必须获得译文和对应Reference之间的word alignment。这里我们使用GIZA++来完成这个任务。<br>
We thus run GIZA++ on all posiible paraphrase combinations together with the reference-translation  paraphrase combinations together with the reference-translation pairs we need to align and then extract alignments only for the sentences of interest.<br>
#### Tectogrammatical Parsing
使用Treex framework（Popel and ... 2010）来实现 tagging , parsing and tectogrammatical annotation.<br>
一个句子的tectogrammatical annotation是一个依赖树，其中的每个节点，只表示对应的具体的内容的单词。节点对应的主要的label是tectogrammatical leamma(承认这个没看懂，想看懂还是得去翻一下paper啊)。Tectogrammatical annotation of sentence is a dependency tree, in which only content words are represented by nodes. The main label of the node is a tectogrammatical lemma - mostly the same as the morphological lemma, sometimes combined with a function word in c ase it changes its meaning.<br>
Other function words and grammatical features of the words are expressed by other attributes of the tectogrammatical node. <br>
An example of a pair of tectogrammatical trees is provided in Figure 1. The main attributes are:
- tectogrammatical lemma (t-lemma): the lexical value of the node,
- functor: the semantic value of the syntac- tic dependency relation. Functors express the functions of individual modifications in the sentence, e.g. ACT (Actor), PAT (Pa- tient), ADDR (Addressee), LOC (Location), MANN (Manner),
- sempos: semantic part of speech: n (noun), adj (adjective), v (verb), or adv (adverbial),
- formeme: morphosyntactic form of the node. The formeme includes for example preposi- tions and cases of the nouns, e.g. n:jako+1 for nominative case with preposition jako.
- grammatemes: tectogrammatical counter- parts of morphological categories, such as number,gender,person,negation,modality, aspect, etc.
![tectogrammatical lemmam](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/tectogrammatical_lemma.png)
#### Linear Regression Training
We collect 83 various features based on match- ing tectogrammatical attributes computed on all nodes or a subsets defined by particular seman- tic part-of-speech tags. To this set of features, we add two BLEU scores (Papineni et al., 2002) computed on forms and on lemmas and two chrF3 scores (Popovic, 2015) computed on trigrams and sixgrams, so we have 87 features in total.
![linear Regression Training](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/LRT.png)
### Language Universal AutoDA
We have seen that deep-syntactic features help to train an automatic metric with higher correlation for Czech. Even though we have no similar tools for other languages so far, we try to extract simi- lar features for them as well. The source code is available online. 
#### Universal Parsing
We use Universal Dependencies (UD) by Nivre et al. (2016b), a collection of treebanks in a com- mon annotation style, where all our testing lan- guages are present – version 1.3 covers 40 lan- guages (Nivre et al., 2016a). For syntactic anal- ysis, we use UDPipe by Straka et al. (2016), a to- kenizer, tagger, and parser in one tool, which is trained on UD. The UD tagset consists of 17 POS tags; the big advantage is that the tagset is the same for all the languages and therefore we can easily extract e.g. content words, prepositional phrases, etc.
#### Monolingual Alignment
Unlike from Czech, we did not known about the existing corpus of paraphrases available across other languages, so we used a simple monolin- gual aligner based on word similarities and rel- ative positions in the sentence. Our implemen- tation is inspired by the heuristic Monolingual Greedy Aligner written by Martin Popel (Rosa et al., 2012), which is available in the Treex frame- work.<br>
First, we compute scores for all possible align- ment connections between tokens of the reference and translated sentence:
$ score(i,j)=w_1JaroWinkler(W_i^t,W_j^r) +w_2I(T_i^t =T_j^r)+ w_3(1 − |(i/len(t) − j/len(r)|),$<br>
where JaroWinkler$(W_i^t , W_j^r )$ defines similarity between the given words (Winkler, 1990), $I(T_i^t = T_j^r)$ is a binary indicator testing the identity of POS tags, and (1−|(i/len(t)−j/len(r)|) tells us how close are the two words according to their relative positions in the sentences. The weights were setmanuallytow1 = 8,w2 = 3,andw3 = 3;
#### Extracting Features
We distinguish content words from function ones by the POS tag. The tags for nouns (NOUN, PROPN), verbs (VERB), adjectives (ADJ), and adverbs (ADV) correspond more or less to content words. Then there are pronouns (PRON), symbols (SYM), and other (X), which may be sometimes content words as well, but we do not count them. The rest of POS tags represent function words.
Now, using the alignment links and the content words, we can compute numbers of matching con- tent word forms and matching content word lem- mas. The universal annotations contains also mor- phological features of words: case, number, tense, etc. Therefore, we also create equivalents of tec- togrammatical formemes or grammatemes. Our features can thus check for instance the percent- age of aligned words with matching morphologi- cal number or tense.

后面的大致看了一下，感觉我会全部用复制粘贴的，所以还是算了吧。